In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName("Google play store Review analysis Using Pyspark").getOrCreate()

Loading data

In [4]:
df = spark.read.load('C:\e drive\Personal\projects\Pyspark\google playstore project\data\googleplaystore.csv',format = 'csv',sep=',',header='true',escape='"',inferschema='true')

Number of records

In [5]:
df.count()

10841

In [6]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



Checking Schema

In [7]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning

In [8]:
df = df.drop("size","content Rating","Last Updated","Android Ver","Current Ver")

In [9]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [10]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [11]:
# Reviews is string format so convert into int ,similarly correct installs,price columns also

In [12]:
from pyspark.sql.functions import regexp_replace , col

df = df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
.withColumn("Price",col("Price").cast(IntegerType()))

In [13]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [14]:
df.show(2) # + symbols in Installs is removed, all fields are corrected

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



Creating Temporary view


In [21]:
df.createOrReplaceTempView("apps")

In [22]:
spark.sql("select * from apps").show()  #using sql commands

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

# Problem - 1

# Find the top 5 rated apps from the data

In [35]:
spark.sql("Select App ,sum(Reviews) from apps group by App order by sum(Reviews) desc limit 5").show()

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
+--------------------+------------+



# Top 5 installed apps

In [36]:
spark.sql("Select App, sum(Installs) from apps group by 1 order by 2 desc limit 5").show()

+--------------+-------------+
|           App|sum(Installs)|
+--------------+-------------+
|Subway Surfers|   6000000000|
| Google Photos|   4000000000|
|      Hangouts|   4000000000|
|     Instagram|   4000000000|
|  Google Drive|   4000000000|
+--------------+-------------+



# Category wise Distribution

In [34]:
spark.sql("Select Category, sum(Installs) from apps group by 1 order by 2 desc limit 5").show()

+-------------+-------------+
|     Category|sum(Installs)|
+-------------+-------------+
|         GAME|  35086024415|
|COMMUNICATION|  32647276251|
| PRODUCTIVITY|  14176091369|
|       SOCIAL|  14069867902|
|        TOOLS|  11452771915|
+-------------+-------------+



# Top paid apps

In [41]:
spark.sql("Select App,sum(Price) from apps where Type ='Paid' group by 1 order by 2 desc limit 5").show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|       400|
|   I Am Rich Premium|       399|
|           I am Rich|       399|
|most expensive ap...|       399|
|      I am Rich Plus|       399|
+--------------------+----------+

